In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

## Modelling
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score, learning_curve
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn import metrics


In [2]:
frequency_domain_features_train = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Train Data/frequency_domain_features_train.csv')
heart_rate_non_linear_features_train = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Train Data/heart_rate_non_linear_features_train.csv')
time_domain_features_train = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Train Data/time_domain_features_train.csv')

In [3]:
print(frequency_domain_features_train.shape)
print(heart_rate_non_linear_features_train.shape)
print(time_domain_features_train.shape)

(369289, 12)
(369289, 7)
(369289, 20)


In [5]:
data_train_temp = pd.merge(frequency_domain_features_train,heart_rate_non_linear_features_train, on = 'uuid')
print(data_train_temp.shape)

(369289, 18)


In [6]:
data_train = pd.merge(data_train_temp,time_domain_features_train, on = 'uuid')
print(data_train.shape)

(369289, 37)


In [7]:
data_train.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
0,89df2855-56eb-4706-a23b-b39363dd605a,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485,2,no stress,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218
1,80c795e4-aa56-4cc0-939c-19634b89cbb2,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711,2,interruption,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286
2,c2d5d102-967c-487d-88f2-8b005a449f3e,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315,2,interruption,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813
3,37eabc44-1349-4040-8896-0d113ad4811f,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688,2,no stress,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138
4,aa777a6a-7aa3-4f6e-aced-70f8691dd2b7,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612,2,no stress,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252


In [8]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369289 entries, 0 to 369288
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   uuid               369289 non-null  object 
 1   VLF                369289 non-null  float64
 2   VLF_PCT            369289 non-null  float64
 3   LF                 369289 non-null  float64
 4   LF_PCT             369289 non-null  float64
 5   LF_NU              369289 non-null  float64
 6   HF                 369289 non-null  float64
 7   HF_PCT             369289 non-null  float64
 8   HF_NU              369289 non-null  float64
 9   TP                 369289 non-null  float64
 10  LF_HF              369289 non-null  float64
 11  HF_LF              369289 non-null  float64
 12  SD1                369289 non-null  float64
 13  SD2                369289 non-null  float64
 14  sampen             369289 non-null  float64
 15  higuci             369289 non-null  float64
 16  da

In [9]:
df_train = data_train.drop(['uuid'],axis = 1)

In [10]:
df_train.describe()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
count,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.0,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,3.692890e+05,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000
mean,2199.580170,64.289242,946.530252,34.095182,95.566718,39.245603,1.615576,4.433282,3185.356025,115.977200,0.048506,10.593708,154.178997,2.062471,1.182292,2.0,846.650104,841.965890,109.352531,14.977498,14.976767,7.396597,73.941824,9.841143,0.866001,0.523235,0.041628,-1.756587e-06,-0.000465,0.018571,0.009701,0.009701,2.006817,0.523235,0.041628
std,1815.773422,16.774844,574.171780,16.040290,4.123365,45.398869,1.761073,4.123365,1923.227187,360.855129,0.049238,2.914795,109.170222,0.206999,0.062192,0.0,124.603984,132.321005,77.117025,4.120766,4.120768,5.143834,10.337453,8.195574,0.990189,1.790348,0.699522,1.630256e-04,0.000868,0.005455,0.003897,0.003897,0.375845,1.790348,0.699522
min,159.480176,19.031219,90.048557,2.165119,69.879083,0.061783,0.002150,0.012825,377.692795,2.319952,0.000128,3.911344,38.307745,0.434576,1.033984,2.0,547.492221,517.293295,27.233947,5.529742,5.529630,2.660381,48.737243,0.000000,0.000000,-1.894820,-2.136278,-1.233914e-03,-0.004425,0.008987,0.003220,0.003220,1.169342,-1.894820,-2.136278
25%,1001.189280,52.909877,545.449386,22.305936,93.645734,10.720312,0.346803,1.228054,1828.147788,14.737458,0.012433,8.368340,90.326864,2.032977,1.139929,2.0,760.228533,755.750735,64.205641,11.830959,11.830671,4.541896,66.715776,3.666667,0.000000,-0.352783,-0.359291,-7.281695e-05,-0.000917,0.014261,0.006984,0.006984,1.749801,-0.352783,-0.359291
50%,1667.903111,66.350237,782.716291,32.047025,96.643140,24.841938,1.039513,3.356860,2796.856587,28.789747,0.034735,10.196621,116.221063,2.134214,1.174293,2.0,822.951438,819.689595,82.608243,14.415918,14.415388,5.952112,74.217809,7.600000,0.466667,0.040736,-0.060966,-9.330777e-07,-0.000312,0.017318,0.008691,0.008691,1.934416,0.040736,-0.060966
75%,2654.121052,76.825032,1201.432256,44.647115,98.771946,45.272368,2.245115,6.354266,4052.260157,80.429614,0.067854,12.679005,166.764850,2.181929,1.223621,2.0,924.117422,916.821570,118.237002,17.927144,17.924839,7.919841,80.334937,13.333333,1.466667,0.722833,0.282417,6.911667e-05,0.000131,0.021827,0.011460,0.011460,2.221232,0.722833,0.282417
max,12617.977191,97.738848,3291.548112,77.928847,99.987175,364.486936,13.095664,30.120917,13390.684098,7796.443096,0.431043,18.836107,796.852945,2.234841,1.361219,2.0,1322.016957,1653.122250,563.486949,26.629477,26.629392,54.523950,113.752309,39.400000,5.466667,64.088107,6.777800,1.244098e-03,0.002095,0.036571,0.026955,0.026955,3.724134,64.088107,6.777800


In [11]:
def missing_check(df):
    total = df.isnull().sum().sort_values(ascending=False)   # total number of null values
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)  # percentage of values that are null
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])  # putting the above two together
    return missing_data # return the dataframe
missing_check(df_train)

,Total,Percent
SKEW_REL_RR,0,0.0
KURT_REL_RR,0,0.0
datasetId,0,0.0
higuci,0,0.0
sampen,0,0.0
SD2,0,0.0
SD1,0,0.0
HF_LF,0,0.0
LF_HF,0,0.0
TP,0,0.0


In [12]:
## User defined function to calculate statistics for the attributes
def stats_measure(x):
  print('Number of Unique Values')
  print(x.nunique())
  print(" ")
  print('Summary of the attribute')
  print(x.describe())
  print(" ")
  print('Skewness of the attribute')
  if x.dtype=='O':
    print('Categorical Variable')
  else:
    print(x.skew())
  print(" ")
  print('Kurtosis of the attribute')
  if x.dtype=='O':
    print('Categorical Variable')
  else:
    print(x.kurt())   


In [13]:
for p in df_train.columns:
  print('-'*100)  
  print(f'Statistical Measures of the attribute: ',p)
  col = df_train[p]
  stats_measure(col)
  print('-'*100)


----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  VLF
Number of Unique Values
369289
 
Summary of the attribute
count    369289.000000
mean       2199.580170
std        1815.773422
min         159.480176
25%        1001.189280
50%        1667.903111
75%        2654.121052
max       12617.977191
Name: VLF, dtype: float64
 
Skewness of the attribute
1.9607349536265608
 
Kurtosis of the attribute
4.493563049525889
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  VLF_PCT
Number of Unique Values
369289
 
Summary of the attribute
count    369289.000000
mean         64.289242
std          16.774844
min          19.031219
25%          52.909877
50%          66.350237
75%          76.825032
max          97.738848
Na

Name: higuci, dtype: float64
 
Skewness of the attribute
0.335008119436041
 
Kurtosis of the attribute
-0.2788429775301653
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  datasetId
Number of Unique Values
1
 
Summary of the attribute
count    369289.0
mean          2.0
std           0.0
min           2.0
25%           2.0
50%           2.0
75%           2.0
max           2.0
Name: datasetId, dtype: float64
 
Skewness of the attribute
0
 
Kurtosis of the attribute
0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  condition
Number of Unique Values
3
 
Summary of the attribute
count        369289
unique     

count    3.692890e+05
mean    -1.756587e-06
std      1.630256e-04
min     -1.233914e-03
25%     -7.281695e-05
50%     -9.330777e-07
75%      6.911667e-05
max      1.244098e-03
Name: MEAN_REL_RR, dtype: float64
 
Skewness of the attribute
0.11279553671256196
 
Kurtosis of the attribute
5.215669435399746
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  MEDIAN_REL_RR
Number of Unique Values
25041
 
Summary of the attribute
count    369289.000000
mean         -0.000465
std           0.000868
min          -0.004425
25%          -0.000917
50%          -0.000312
75%           0.000131
max           0.002095
Name: MEDIAN_REL_RR, dtype: float64
 
Skewness of the attribute
-0.9480094763120768
 
Kurtosis of the attribute
1.26952057309774
----------------------------------------------------------------------

In [14]:
# List the numerical and categorical columns
numeric_cols = df_train.select_dtypes(include = [np.number]).columns.tolist()
categorical_cols = df_train.select_dtypes(include = [np.object]).columns.tolist()
print('The numeric attributes are:', numeric_cols)
print('The categorical attributes are:', categorical_cols)

The numeric attributes are: ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen', 'higuci', 'datasetId', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR']
The categorical attributes are: ['condition']


In [15]:
df_train['condition'].value_counts()

no stress        200082
interruption     105150
time pressure     64057
Name: condition, dtype: int64

In [16]:
dummy_var = pd.get_dummies(df_train['condition'],drop_first = True)
dummy_var.head()

,no stress,time pressure
0,1,0
1,0,0
2,0,0
3,1,0
4,1,0


In [17]:
df_train = pd.concat([df_train, dummy_var], axis = 1)
df_train = df_train.drop('condition', axis = 1)

In [18]:
df_train.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,no stress,time pressure
0,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485,2,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218,1,0
1,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711,2,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286,0,0
2,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315,2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813,0,0
3,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688,2,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138,1,0
4,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612,2,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252,1,0


In [19]:
df_train_imputed = df_train.copy(deep = True)

In [20]:
for col in numeric_cols:
    Q3 = df_train_imputed[col].quantile(0.75)
    Q1 = df_train_imputed[col].quantile(0.25)
    IQR = Q3 - Q1
    upper_lim = Q3 + (3 * IQR)
    lower_lim = Q1 - (3 * IQR)
    df_train_imputed.loc[(df_train_imputed[col] < lower_lim) | (df_train_imputed[col] > upper_lim), col] = df_train_imputed[col].median()

#df_train_imputed.isnull().sum()

In [21]:
# Using SimpleImputer to fill missing values by median
print('Descriptive Stats before handling outliers: \n', '--'*30)
display(df_train.describe().T)

print('Descriptive Stats after handling outliers: \n', '--'*30)
display(df_train_imputed.describe().T)


Descriptive Stats before handling outliers: 
 ------------------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
VLF,369289.0,2199.580170,1815.773422,159.480176,1001.189280,1.667903e+03,2654.121052,12617.977191
VLF_PCT,369289.0,64.289242,16.774844,19.031219,52.909877,6.635024e+01,76.825032,97.738848
LF,369289.0,946.530252,574.171780,90.048557,545.449386,7.827163e+02,1201.432256,3291.548112
LF_PCT,369289.0,34.095182,16.040290,2.165119,22.305936,3.204703e+01,44.647115,77.928847
LF_NU,369289.0,95.566718,4.123365,69.879083,93.645734,9.664314e+01,98.771946,99.987175
HF,369289.0,39.245603,45.398869,0.061783,10.720312,2.484194e+01,45.272368,364.486936
HF_PCT,369289.0,1.615576,1.761073,0.002150,0.346803,1.039513e+00,2.245115,13.095664
HF_NU,369289.0,4.433282,4.123365,0.012825,1.228054,3.356860e+00,6.354266,30.120917
TP,369289.0,3185.356025,1923.227187,377.692795,1828.147788,2.796857e+03,4052.260157,13390.684098
LF_HF,369289.0,115.977200,360.855129,2.319952,14.737458,2.878975e+01,80.429614,7796.443096


Descriptive Stats after handling outliers: 
 ------------------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
VLF,369289.0,2005.718669,1426.877298,159.480176,1001.189280,1.667903e+03,2496.617475,7612.783844
VLF_PCT,369289.0,64.289242,16.774844,19.031219,52.909877,6.635024e+01,76.825032,97.738848
LF,369289.0,944.821855,571.037542,90.048557,545.449386,7.827163e+02,1199.536481,3169.378593
LF_PCT,369289.0,34.095182,16.040290,2.165119,22.305936,3.204703e+01,44.647115,77.928847
LF_NU,369289.0,95.698710,3.805403,78.269762,93.729698,9.664314e+01,98.771946,99.987175
HF,369289.0,33.564026,32.819622,0.061783,10.720312,2.484194e+01,41.551803,148.921411
HF_PCT,369289.0,1.508332,1.508504,0.002150,0.346803,1.039513e+00,2.137243,7.938278
HF_NU,369289.0,4.301290,3.805403,0.012825,1.228054,3.356860e+00,6.270302,21.730238
TP,369289.0,3147.309366,1836.098125,377.692795,1828.147788,2.796857e+03,4024.752713,10724.569836
LF_HF,369289.0,47.087608,53.973996,2.319952,14.737458,2.878975e+01,52.132280,277.505613


### Actual test set


In [22]:
frequency_domain_features_test = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Test Data/frequency_domain_features_test.csv')
heart_rate_non_linear_features_test = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Test Data/heart_rate_non_linear_features_test.csv')
time_domain_features_test = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Test Data/time_domain_features_test.csv')

In [23]:
print(frequency_domain_features_test.shape)
print(heart_rate_non_linear_features_test.shape)
print(time_domain_features_test.shape)

(41033, 12)
(41033, 7)
(41033, 19)


In [24]:
data_test_temp = pd.merge(frequency_domain_features_test,heart_rate_non_linear_features_test, on = 'uuid')
print(data_test_temp.shape)

(41033, 18)


In [25]:
data_test = pd.merge(data_test_temp,time_domain_features_test, on = 'uuid')
print(data_test.shape)

(41033, 36)


In [26]:
data_test.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
0,62b75db5-bc40-4c8f-9166-daf0efcab4c2,1868.532278,76.511189,570.643114,23.366245,99.478197,2.993254,0.122565,0.521803,2442.168645,190.643094,0.005245,8.347898,115.862444,2.209659,1.100715,2,time pressure,934.665288,939.03173,82.139495,11.801781,11.801772,6.959924,3.933333,0.133333,-0.680262,-0.233075,0.000016,0.000288,0.012933,0.004578,0.004578,2.825038,-0.680262,-0.233075
1,a99549ad-3eb6-4413-bc90-9053e7f7e684,568.742845,26.301350,1553.971621,71.862973,97.509212,39.694850,1.835677,2.490788,2162.409316,39.147940,0.025544,14.542096,77.118903,2.186132,1.290615,2,no stress,817.062380,816.33879,55.492332,20.558810,20.558768,2.699200,24.600000,0.533333,-0.034454,-0.051689,0.000040,-0.002749,0.025148,0.013921,0.013921,1.806517,-0.034454,-0.051689
2,cb573d3a-c767-4556-b32e-ad8c08ded214,2101.871207,75.836461,655.175895,23.639042,97.829386,14.536877,0.524497,2.170614,2771.583978,45.069921,0.022188,9.799336,125.044377,2.051571,1.226663,2,no stress,876.762022,894.19889,88.690820,13.853737,13.853730,6.401942,7.066667,0.533333,-0.206953,-0.589940,0.000010,-0.000172,0.015533,0.008149,0.008149,1.906180,-0.206953,-0.589940
3,47a0c6de-2aef-4ac3-997d-252fa6fd07f1,5757.544433,90.562305,592.913021,9.326123,98.817806,7.093235,0.111572,1.182194,6357.550689,83.588517,0.011963,11.639185,302.029812,2.080910,1.085143,2,time pressure,1038.640693,998.91429,213.725850,16.457194,16.454801,12.986774,10.800000,1.866667,-0.820407,0.487198,-0.000238,-0.000464,0.016882,0.007587,0.007587,2.225116,-0.820407,0.487198
4,de3fd54f-c74e-4fe8-bf2a-7a127f68b312,964.696325,70.256575,374.939530,27.305968,91.805057,33.468834,2.437457,8.194943,1373.104689,11.202647,0.089265,7.266567,72.579248,2.068728,1.252547,2,interruption,774.548508,778.90508,51.577855,10.273114,10.273049,5.020664,2.000000,0.066667,1.738453,-0.005082,0.000054,-0.000006,0.013479,0.007811,0.007811,1.725606,1.738453,-0.005082


In [27]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41033 entries, 0 to 41032
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   uuid               41033 non-null  object 
 1   VLF                41033 non-null  float64
 2   VLF_PCT            41033 non-null  float64
 3   LF                 41033 non-null  float64
 4   LF_PCT             41033 non-null  float64
 5   LF_NU              41033 non-null  float64
 6   HF                 41033 non-null  float64
 7   HF_PCT             41033 non-null  float64
 8   HF_NU              41033 non-null  float64
 9   TP                 41033 non-null  float64
 10  LF_HF              41033 non-null  float64
 11  HF_LF              41033 non-null  float64
 12  SD1                41033 non-null  float64
 13  SD2                41033 non-null  float64
 14  sampen             41033 non-null  float64
 15  higuci             41033 non-null  float64
 16  datasetId          410

In [28]:
df_test = data_test.drop(['uuid'],axis = 1)

In [29]:
df_test.describe()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
count,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.0,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,4.103300e+04,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000,41033.000000
mean,2196.457487,64.221039,948.851848,34.168102,95.586613,39.012113,1.610860,4.413387,3184.321449,117.181622,0.048261,10.602260,153.475051,2.063608,1.182641,2.0,846.856304,842.112932,108.856642,14.989588,14.988858,7.369326,9.888347,0.862545,0.520719,0.044934,-2.872766e-08,-0.000468,0.018561,0.009690,0.009690,2.008090,0.520719,0.044934
std,1815.262687,16.838147,577.413541,16.105461,4.102133,44.994281,1.750709,4.102133,1923.544936,368.039153,0.048859,2.927793,108.442308,0.207427,0.062489,0.0,124.422504,131.976225,76.602112,4.139121,4.139143,5.146460,8.262497,0.986774,1.791907,0.694370,1.620372e-04,0.000865,0.005451,0.003886,0.003886,0.377263,1.791907,0.694370
min,160.869730,19.301549,92.663980,2.163917,70.682584,0.062809,0.002211,0.012971,378.053805,2.410942,0.000130,3.912661,38.454442,0.435664,1.034420,2.0,547.483802,517.293295,27.338606,5.531588,5.531493,2.664721,0.000000,0.000000,-1.894831,-2.136234,-1.275240e-03,-0.004425,0.008987,0.003220,0.003220,1.186548,-1.894831,-2.136234
25%,994.970318,52.721914,544.688588,22.299698,93.659224,10.551975,0.338246,1.205268,1820.982721,14.770940,0.012200,8.370315,90.236880,2.034956,1.140058,2.0,760.387193,755.686825,64.132798,11.833717,11.833463,4.517620,3.600000,0.000000,-0.352526,-0.351531,-7.011447e-05,-0.000919,0.014237,0.006972,0.006972,1.751317,-0.352526,-0.351531
50%,1665.049572,66.240263,783.452494,32.171726,96.658088,24.654848,1.034102,3.341912,2799.392704,28.922987,0.034575,10.207964,116.339025,2.135147,1.174465,2.0,822.611612,819.624875,82.710729,14.431598,14.431425,5.934226,7.600000,0.466667,0.039643,-0.053997,2.305116e-07,-0.000312,0.017305,0.008698,0.008698,1.935984,0.039643,-0.053997
75%,2657.910801,76.845957,1210.276983,44.754617,98.794732,45.004948,2.237060,6.340776,4055.490025,81.969112,0.067700,12.710279,166.211038,2.182363,1.224328,2.0,925.053520,917.984880,117.822094,17.969971,17.969053,7.906338,13.333333,1.400000,0.711312,0.287011,7.000961e-05,0.000125,0.021839,0.011451,0.011451,2.220445,0.711312,0.287011
max,12427.177612,97.739588,3290.093757,77.522361,99.987029,360.877726,13.087775,29.317416,13249.507794,7708.369846,0.414776,18.796168,796.293973,2.234642,1.360587,2.0,1321.597359,1653.122250,563.092252,26.572991,26.572927,54.438959,39.400000,5.400000,54.650655,6.656766,1.211792e-03,0.002079,0.036575,0.026861,0.026861,3.705506,54.650655,6.656766


In [30]:
def missing_check(df):
    total = df.isnull().sum().sort_values(ascending=False)   # total number of null values
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)  # percentage of values that are null
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])  # putting the above two together
    return missing_data # return the dataframe
missing_check(df_test)

,Total,Percent
SKEW_REL_RR,0,0.0
TP,0,0.0
higuci,0,0.0
sampen,0,0.0
SD2,0,0.0
SD1,0,0.0
HF_LF,0,0.0
LF_HF,0,0.0
HF_NU,0,0.0
condition,0,0.0


In [31]:
## User defined function to calculate statistics for the attributes
def stats_measure(x):
  print('Number of Unique Values')
  print(x.nunique())
  print(" ")
  print('Summary of the attribute')
  print(x.describe())
  print(" ")
  print('Skewness of the attribute')
  if x.dtype=='O':
    print('Categorical Variable')
  else:
    print(x.skew())
  print(" ")
  print('Kurtosis of the attribute')
  if x.dtype=='O':
    print('Categorical Variable')
  else:
    print(x.kurt())   


In [32]:
for p in df_test.columns:
  print('-'*100)  
  print(f'Statistical Measures of the attribute: ',p)
  col = df_test[p]
  stats_measure(col)
  print('-'*100)


----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  VLF
Number of Unique Values
41033
 
Summary of the attribute
count    41033.000000
mean      2196.457487
std       1815.262687
min        160.869730
25%        994.970318
50%       1665.049572
75%       2657.910801
max      12427.177612
Name: VLF, dtype: float64
 
Skewness of the attribute
1.9676334696591826
 
Kurtosis of the attribute
4.57957065835418
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  VLF_PCT
Number of Unique Values
41033
 
Summary of the attribute
count    41033.000000
mean        64.221039
std         16.838147
min         19.301549
25%         52.721914
50%         66.240263
75%         76.845957
max         97.739588
Name: VLF_PCT, dtype:

Name: pNN25, dtype: float64
 
Skewness of the attribute
1.1847342097958065
 
Kurtosis of the attribute
0.7462756233505878
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  pNN50
Number of Unique Values
81
 
Summary of the attribute
count    41033.000000
mean         0.862545
std          0.986774
min          0.000000
25%          0.000000
50%          0.466667
75%          1.400000
max          5.400000
Name: pNN50, dtype: float64
 
Skewness of the attribute
1.2699529249835075
 
Kurtosis of the attribute
1.069007190710861
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  KURT
Number of Unique Values
41033
 


In [33]:
# List the numerical and categorical columns
numeric_cols = df_test.select_dtypes(include = [np.number]).columns.tolist()
categorical_cols = df_test.select_dtypes(include = [np.object]).columns.tolist()
print('The numeric attributes are:', numeric_cols)
print('The categorical attributes are:', categorical_cols)

The numeric attributes are: ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen', 'higuci', 'datasetId', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR']
The categorical attributes are: ['condition']


In [34]:
df_test['condition'].value_counts()

no stress        22158
interruption     11782
time pressure     7093
Name: condition, dtype: int64

In [35]:
dummy_var = pd.get_dummies(df_test['condition'],drop_first = True)
dummy_var.head()

,no stress,time pressure
0,0,1
1,1,0
2,1,0
3,0,1
4,0,0


In [36]:
df_test = pd.concat([df_test, dummy_var], axis = 1)
df_test = df_test.drop('condition', axis = 1)

In [37]:
df_test.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,no stress,time pressure
0,1868.532278,76.511189,570.643114,23.366245,99.478197,2.993254,0.122565,0.521803,2442.168645,190.643094,0.005245,8.347898,115.862444,2.209659,1.100715,2,934.665288,939.03173,82.139495,11.801781,11.801772,6.959924,3.933333,0.133333,-0.680262,-0.233075,0.000016,0.000288,0.012933,0.004578,0.004578,2.825038,-0.680262,-0.233075,0,1
1,568.742845,26.301350,1553.971621,71.862973,97.509212,39.694850,1.835677,2.490788,2162.409316,39.147940,0.025544,14.542096,77.118903,2.186132,1.290615,2,817.062380,816.33879,55.492332,20.558810,20.558768,2.699200,24.600000,0.533333,-0.034454,-0.051689,0.000040,-0.002749,0.025148,0.013921,0.013921,1.806517,-0.034454,-0.051689,1,0
2,2101.871207,75.836461,655.175895,23.639042,97.829386,14.536877,0.524497,2.170614,2771.583978,45.069921,0.022188,9.799336,125.044377,2.051571,1.226663,2,876.762022,894.19889,88.690820,13.853737,13.853730,6.401942,7.066667,0.533333,-0.206953,-0.589940,0.000010,-0.000172,0.015533,0.008149,0.008149,1.906180,-0.206953,-0.589940,1,0
3,5757.544433,90.562305,592.913021,9.326123,98.817806,7.093235,0.111572,1.182194,6357.550689,83.588517,0.011963,11.639185,302.029812,2.080910,1.085143,2,1038.640693,998.91429,213.725850,16.457194,16.454801,12.986774,10.800000,1.866667,-0.820407,0.487198,-0.000238,-0.000464,0.016882,0.007587,0.007587,2.225116,-0.820407,0.487198,0,1
4,964.696325,70.256575,374.939530,27.305968,91.805057,33.468834,2.437457,8.194943,1373.104689,11.202647,0.089265,7.266567,72.579248,2.068728,1.252547,2,774.548508,778.90508,51.577855,10.273114,10.273049,5.020664,2.000000,0.066667,1.738453,-0.005082,0.000054,-0.000006,0.013479,0.007811,0.007811,1.725606,1.738453,-0.005082,0,0


In [38]:
df_test_imputed = df_test.copy(deep = True)

In [39]:
for col in numeric_cols:
    Q3 = df_test_imputed[col].quantile(0.75)
    Q1 = df_test_imputed[col].quantile(0.25)
    IQR = Q3 - Q1
    upper_lim = Q3 + (3 * IQR)
    lower_lim = Q1 - (3 * IQR)
    df_test_imputed.loc[(df_test_imputed[col] < lower_lim) | (df_test_imputed[col] > upper_lim), col] = np.nan
df_test_imputed.isnull().sum()

VLF                  1083
VLF_PCT                 0
LF                     13
LF_PCT                  0
LF_NU                 235
HF                   1280
HF_PCT                474
HF_NU                 235
TP                    184
LF_HF                3259
HF_LF                 448
SD1                     0
SD2                  2375
sampen               1529
higuci                  0
datasetId               0
MEAN_RR                 0
MEDIAN_RR              83
SDRR                 2372
RMSSD                   0
SDSD                    0
SDRR_RMSSD           1734
pNN25                   0
pNN50                   0
KURT                 1874
SKEW                  612
MEAN_REL_RR           881
MEDIAN_REL_RR          79
SDRR_REL_RR             0
RMSSD_REL_RR           85
SDSD_REL_RR            85
SDRR_RMSSD_REL_RR      22
KURT_REL_RR          1874
SKEW_REL_RR           612
no stress               0
time pressure           0
dtype: int64

In [40]:
for col in numeric_cols:
    Q3 = df_test_imputed[col].quantile(0.75)
    Q1 = df_test_imputed[col].quantile(0.25)
    IQR = Q3 - Q1
    upper_lim = Q3 + (3 * IQR)
    lower_lim = Q1 - (3 * IQR)
    df_test_imputed.loc[(df_test_imputed[col] < lower_lim) | (df_test_imputed[col] > upper_lim), col] = df_test_imputed[col].median()


In [41]:
# Using SimpleImputer to fill missing values by median
print('Descriptive Stats before handling outliers: \n', '--'*30)
display(df_test.describe().T)

print('Descriptive Stats after handling outliers: \n', '--'*30)
display(df_test_imputed.describe().T)


Descriptive Stats before handling outliers: 
 ------------------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
VLF,41033.0,2.196457e+03,1815.262687,160.869730,994.970318,1.665050e+03,2657.910801,12427.177612
VLF_PCT,41033.0,6.422104e+01,16.838147,19.301549,52.721914,6.624026e+01,76.845957,97.739588
LF,41033.0,9.488518e+02,577.413541,92.663980,544.688588,7.834525e+02,1210.276983,3290.093757
LF_PCT,41033.0,3.416810e+01,16.105461,2.163917,22.299698,3.217173e+01,44.754617,77.522361
LF_NU,41033.0,9.558661e+01,4.102133,70.682584,93.659224,9.665809e+01,98.794732,99.987029
HF,41033.0,3.901211e+01,44.994281,0.062809,10.551975,2.465485e+01,45.004948,360.877726
HF_PCT,41033.0,1.610860e+00,1.750709,0.002211,0.338246,1.034102e+00,2.237060,13.087775
HF_NU,41033.0,4.413387e+00,4.102133,0.012971,1.205268,3.341912e+00,6.340776,29.317416
TP,41033.0,3.184321e+03,1923.544936,378.053805,1820.982721,2.799393e+03,4055.490025,13249.507794
LF_HF,41033.0,1.171816e+02,368.039153,2.410942,14.770940,2.892299e+01,81.969112,7708.369846


Descriptive Stats after handling outliers: 
 ------------------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
VLF,39950.0,1985.646378,1395.717131,160.869730,982.277629,1.625906e+03,2513.196124,7207.127755
VLF_PCT,41033.0,64.221039,16.838147,19.301549,52.721914,6.624026e+01,76.845957,97.739588
LF,41020.0,948.069120,575.964671,92.663980,544.609021,7.833408e+02,1209.123033,3201.257544
LF_PCT,41033.0,34.168102,16.105461,2.163917,22.299698,3.217173e+01,44.754617,77.522361
LF_NU,40798.0,95.706451,3.811548,78.429557,93.711391,9.668760e+01,98.802196,99.987029
HF,39753.0,32.442429,31.315650,0.062809,10.120750,2.374117e+01,41.259745,138.353671
HF_PCT,40559.0,1.510942,1.515335,0.002211,0.332353,1.016848e+00,2.152899,7.644612
HF_NU,40798.0,4.293549,3.811548,0.012971,1.197804,3.312396e+00,6.288609,21.570443
TP,40849.0,3143.975687,1831.423297,378.053805,1817.192069,2.791058e+03,4031.776906,10622.811820
LF_HF,37774.0,39.455965,39.169475,2.410942,14.126019,2.587785e+01,47.824967,192.952944


In [83]:
X_train = df_train_imputed.drop(['HR'],axis = 1)
y_train = df_train_imputed['HR']
X_test = df_test_imputed.copy(deep = True)

In [84]:
X = df_train_imputed.drop(['HR'],axis = 1)

In [85]:
## Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = pd.DataFrame(X_test, columns = df_test_imputed.columns)

In [86]:
X_train.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,no stress,time pressure
0,0.459869,0.471781,0.112826,-0.418915,0.732263,-0.549715,-0.720771,-0.732263,0.293752,0.332206,-0.728987,0.139927,1.031077,0.354312,-0.302406,0.0,0.309041,0.105193,1.028522,0.140024,0.139927,0.843213,0.157670,-0.335965,-1.183116,0.547897,-1.496913,0.327171,-0.273263,-0.443299,-0.443295,0.367771,-1.183116,0.547897,0.919613,-0.458108
1,0.216239,0.756281,-0.446046,-0.694567,1.050268,-0.958437,-0.953396,-1.050268,-0.076697,-0.339013,-1.027197,-0.488398,-0.264793,0.626967,-1.569039,0.0,0.744562,0.847024,-0.267243,-0.488516,-0.488398,-0.095583,-0.517493,-0.874583,-0.691025,-0.250785,-0.416569,1.252963,-0.842030,-1.280463,-1.280470,2.474453,-0.691025,-0.250785,-1.087414,-0.458108
2,-0.442808,-0.783139,0.618876,0.887761,0.854513,-0.603125,-0.660033,-0.854513,-0.251310,0.874372,-0.844490,0.322393,-0.198719,0.319750,-0.096115,0.0,0.413598,0.522739,-0.198181,0.322217,0.322393,-0.462307,0.393570,-0.672601,0.143072,-1.067415,-0.057452,0.229227,-0.005791,-0.247876,-0.247872,0.324197,0.143072,-1.067415,-1.087414,-0.458108
3,0.283584,0.351185,0.107103,-0.299551,0.663793,-0.468687,-0.648952,-0.663793,0.156425,0.152687,-0.663817,0.180379,-0.070105,0.657115,-0.041875,0.0,0.281755,0.416502,-0.070607,0.180299,0.180379,-0.271778,0.239014,-0.739928,-0.797219,-0.626678,0.861899,1.115580,-0.148392,-0.262529,-0.262525,0.120714,-0.797219,-0.626678,0.919613,-0.458108
4,-0.598890,-1.214372,0.835253,1.256025,0.268381,0.446435,0.219517,-0.268381,-0.286551,-0.326078,-0.280599,1.028181,-0.677928,0.992827,1.082438,0.0,-0.297140,-0.228721,-0.670200,1.028043,1.028181,-1.106659,1.263959,-0.672601,-0.844918,-0.249102,-0.722290,-2.669183,0.943228,0.887489,0.887493,-0.506450,-0.844918,-0.249102,0.919613,-0.458108


In [89]:
X_train.shape, X_test.shape, y_train.shape

((369289, 36), (41033, 36), (369289,))

In [88]:
model=RandomForestRegressor()
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [52]:
from sklearn import metrics
y_pred_RF = model.predict(X_test.fillna(X_test.mean()))
# performance on train data
#print('Performance on training data using RFR:',model.score(X_train,y_train))

In [63]:
data_test['HR'] = y_pred_RF

In [68]:
submission_RF_1 = data_test[['uuid','HR']]

In [69]:
submission_RF_1.to_csv('submission_RF_1.csv',index=False)

#### For 2nd submission

In [70]:
X_train = df_train_imputed.drop(['HR'],axis = 1)
y_train = df_train_imputed['HR']
X_test = df_test.copy(deep = True)

In [71]:
X = df_train_imputed.drop(['HR'],axis = 1)

In [72]:
## Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = pd.DataFrame(X_test, columns = df_test_imputed.columns)

In [73]:
X_train.shape, X_test.shape, y_train.shape

((369289, 36), (41033, 36), (369289,))

In [74]:
model_RF=RandomForestRegressor()
model_RF.fit(X_train, y_train)

RandomForestRegressor()

In [75]:
from sklearn import metrics
y_pred_RF = model_RF.predict(X_test.fillna(X_test.mean()))
# performance on train data
#print('Performance on training data using RFR:',model.score(X_train,y_train))

In [76]:
data_test['HR'] = y_pred_RF

In [77]:
submission_RF_2 = data_test[['uuid','HR']]

In [78]:
submission_RF_2.to_csv('submission_RF_2.csv',index=False)